In [1]:
import numpy as np
from keras.datasets import mnist
from keras.utils import to_categorical
from keras.layers import Dense
from keras.regularizers import l2
from keras.models import Sequential, Model
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, Callback

Using TensorFlow backend.


In [2]:
NUM_CLASSES = 10

In [3]:
(train_x, train_y), (test_x, test_y) = mnist.load_data()

In [4]:
train_x.shape

(60000, 28, 28)

In [5]:
train_x = train_x.reshape(train_x.shape[0], -1)
train_x = train_x / 255.0
train_y = to_categorical(train_y, NUM_CLASSES)

In [6]:
def build_model():
    model = Sequential()
    model.add(Dense(500, activation='relu', kernel_regularizer=l2(0.001), input_shape=(784,)))
    model.add(Dense(NUM_CLASSES, activation='softmax'))
    model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [7]:
model = build_model()

callbacks = [
    EarlyStopping(monitor='acc', patience=1),
    ModelCheckpoint(filepath='model.h5', monitor='val_loss', save_best_only=True)
]

history = model.fit(
    train_x, train_y, batch_size=128, epochs=20, validation_split=0.2, 
    callbacks=callbacks, verbose=2
)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Train on 48000 samples, validate on 12000 samples
Epoch 1/20
 - 2s - loss: 0.5309 - acc: 0.9123 - val_loss: 0.3094 - val_acc: 0.9510
Epoch 2/20
 - 1s - loss: 0.2664 - acc: 0.9547 - val_loss: 0.2185 - val_acc: 0.9637
Epoch 3/20
 - 1s - loss: 0.2115 - acc: 0.9642 - val_loss: 0.1979 - val_acc: 0.9659
Epoch 4/20
 - 1s - loss: 0.1881 - acc: 0.9675 - val_loss: 0.1864 - val_acc: 0.9672
Epoch 5/20
 - 1s - loss: 0.1731 - acc: 0.9703 - val_loss: 0.1837 - val_acc: 0.9658
Epoch 6/20
 - 1s - loss: 0.1638 - acc: 0.9718 - val_loss: 0.1905 - val_acc: 0.9628
Epoch 7/20
 - 1s - loss: 0.1585 - acc: 0.9726 - val_loss: 0.1566 - val_acc: 0.9740
Epoch 8/20
 - 1s - loss: 0.1506 - acc: 0.9744 - val_loss: 0.1618 - val_acc: 0.9716
Epoch 9/20
 - 1s - loss: 0.1464 - acc: 0.9754 - val_loss: 0.1522 - val_acc: 0.9733
Epoch 10/20
 - 1s - loss: 0.1421 - acc: 0.9757 - val_loss: 0.1515 - val_acc: 0.9744

In [8]:
model = build_model()

callbacks = [
    ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2, verbose=1),
]

history = model.fit(
    train_x, train_y, batch_size=128, epochs=20, validation_split=0.2, 
    callbacks=callbacks, verbose=2
)

Train on 48000 samples, validate on 12000 samples
Epoch 1/20
 - 1s - loss: 0.5294 - acc: 0.9109 - val_loss: 0.3112 - val_acc: 0.9488
Epoch 2/20
 - 1s - loss: 0.2684 - acc: 0.9547 - val_loss: 0.2411 - val_acc: 0.9582
Epoch 3/20
 - 1s - loss: 0.2124 - acc: 0.9621 - val_loss: 0.2013 - val_acc: 0.9642
Epoch 4/20
 - 1s - loss: 0.1878 - acc: 0.9678 - val_loss: 0.1769 - val_acc: 0.9682
Epoch 5/20
 - 1s - loss: 0.1737 - acc: 0.9695 - val_loss: 0.1742 - val_acc: 0.9684
Epoch 6/20
 - 1s - loss: 0.1642 - acc: 0.9712 - val_loss: 0.1873 - val_acc: 0.9621
Epoch 7/20
 - 1s - loss: 0.1574 - acc: 0.9732 - val_loss: 0.1626 - val_acc: 0.9692
Epoch 8/20
 - 1s - loss: 0.1514 - acc: 0.9741 - val_loss: 0.1592 - val_acc: 0.9724
Epoch 9/20
 - 1s - loss: 0.1467 - acc: 0.9755 - val_loss: 0.1589 - val_acc: 0.9715
Epoch 10/20
 - 1s - loss: 0.1418 - acc: 0.9758 - val_loss: 0.1523 - val_acc: 0.9725
Epoch 11/20
 - 1s - loss: 0.1386 - acc: 0.9765 - val_loss: 0.1619 - val_acc: 0.9712
Epoch 12/20
 - 1s - loss: 0.1356 - 

In [9]:
class ActivationLogger(Callback):
    def set_model(self, model):
        self.model = model
        layer_outputs = [layer.output for layer in model.layers]
        self.activations_model = Model(model.input, layer_outputs)
        
    def on_epoch_end(self, epoch, logs=None):
        if self.validation_data is None:
            raise RuntimeError('Requires valid validation_data.')
        validation_sample = self.validation_data[0][0:1]
        activations = self.activations_model.predict(validation_sample)
        with open('activations_at_epohc_{}.npz'.format(epoch), 'wb') as f:
            np.savez(f, *activations)

model = build_model()

callbacks = [
    ActivationLogger(),
]

history = model.fit(
    train_x, train_y, batch_size=128, epochs=10, validation_split=0.2, 
    callbacks=callbacks, verbose=2
)

Train on 48000 samples, validate on 12000 samples
Epoch 1/10
 - 1s - loss: 0.5379 - acc: 0.9092 - val_loss: 0.3223 - val_acc: 0.9469
Epoch 2/10
 - 1s - loss: 0.2708 - acc: 0.9541 - val_loss: 0.2339 - val_acc: 0.9593
Epoch 3/10
 - 1s - loss: 0.2126 - acc: 0.9634 - val_loss: 0.1920 - val_acc: 0.9658
Epoch 4/10
 - 1s - loss: 0.1874 - acc: 0.9672 - val_loss: 0.1766 - val_acc: 0.9681
Epoch 5/10
 - 1s - loss: 0.1723 - acc: 0.9704 - val_loss: 0.1778 - val_acc: 0.9685
Epoch 6/10
 - 1s - loss: 0.1637 - acc: 0.9720 - val_loss: 0.1638 - val_acc: 0.9704
Epoch 7/10
 - 1s - loss: 0.1563 - acc: 0.9729 - val_loss: 0.1609 - val_acc: 0.9715
Epoch 8/10
 - 1s - loss: 0.1503 - acc: 0.9751 - val_loss: 0.1709 - val_acc: 0.9659
Epoch 9/10
 - 1s - loss: 0.1458 - acc: 0.9755 - val_loss: 0.1507 - val_acc: 0.9736
Epoch 10/10
 - 1s - loss: 0.1415 - acc: 0.9760 - val_loss: 0.1625 - val_acc: 0.9714
